# Extract, Transform and Load Spotify Data Set
<hr>

### Dependencies

In [108]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import requests
import pymongo
from pymongo import MongoClient
import json
import numpy as np
import dns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Extract Data Set

In [114]:
path = "../Resources/Variables19082.csv"
data = pd.read_csv(path)

In [115]:
data.head(5)

,User,Emp_type,Gate,Gender,Department,Country,Function,Job_ control_point,Scale,Promotions,Intern_Asig,Age,Seniority,Dependents,Job Code,Comparatio,Status
0,H238709,Regular employee,2,Male,G,India,Operations,F2-ESG,15,1,No,54,1,4,PF10,1.155543,Active
1,HBAO258,Regular employee,2,Male,C,Thailand,Operations,H2-ESG,21,2,No,47,26,1,PC11,1.212336,Active
2,HBAR605,CommuterOut-Halco,2,Male,C,Brunei,Operations,F2-ESG,15,3,Yes,47,24,0,PC10,0.702072,Active
3,HBAO792,Regular employee,1,Male,G,Brunei,Operations,C1-ESG,4,1,Yes,47,27,0,PS03,1.366488,Active
4,HBAO783,Regular employee,2,Male,D,Brunei,Operations,E2-ESG,11,1,Yes,48,25,4,CC06,1.276654,Active


### Transform 

In [116]:
print("We have a total of " + str(len(data)) + " rows in the data set")

We have a total of 1235 rows in the data set


In [117]:
data.dtypes

User                   object
Emp_type               object
Gate                    int64
Gender                 object
Department             object
Country                object
Function               object
Job_ control_point     object
Scale                   int64
Promotions              int64
Intern_Asig            object
Age                     int64
Seniority               int64
Dependents              int64
Job Code               object
Comparatio            float64
Status                 object
dtype: object

### Change date type from obj to float

In [26]:
# data["Date"] = pd.to_datetime(spotify_data["Date"], format="%m/%d/%Y")

In [29]:
# data.dtypes

### Verify NAs and kill them all

In [84]:
data["User"].isna().value_counts()

False    1235
Name: User, dtype: int64

In [85]:
data_wNA = data.dropna()
data_wNA["User"].isna().value_counts()

False    1235
Name: User, dtype: int64

## Load Final HHRR Data Base to MongoDB

In [103]:
pip install dnspython

### Loading Employees Table

In [106]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.HHRRdb
collection = db.Employees
collection.drop()


rows=len(data_wNA)
data=data_wNA
hhrr_dict={}
x=0

for row in range(0,rows):
        
    hhrr_dict={'User':data.iloc[row,0],
           'Emp_type':data.iloc[row,1],
                   'Gate':data.iloc[row,2].astype(float),
                 'Gender':data.iloc[row,3],
             'Department':data.iloc[row,4],
                'Country':data.iloc[row,5],
         'Hyper_function':data.iloc[row,6],
     'Job_ control_point':data.iloc[row,7],
                  'Scale':data.iloc[row,8].astype(float),
             'Promotions':data.iloc[row,9].astype(float),
     'International Asig':data.iloc[row,10],
                    'Age':data.iloc[row,11].astype(float), 
              'Seniority':data.iloc[row,12].astype(float), 
             'Dependants':data.iloc[row,13].astype(float), 
               'Job Code':data.iloc[row,14],
            'Comparation':data.iloc[row,15].astype(float),
              }
    
    collection.insert_one(hhrr_dict)

In [107]:
#pip install pymongo[srv]